# Ground Truth Cleanup Script

This notebook:
1. Loads a ground truth CSV file
2. Fills "NULL" values or empty strings with NaN or None
3. Appends sections from the EDGAR corpus (HuggingFace)
4. Shows bar graphs of non-null values per category
5. Saves the cleaned CSV with timestamp

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from datetime import datetime
import os

## Configuration

In [ ]:
# Path to the ground truth file
GROUND_TRUTH_PATH = "../../data/ground_truth/v3_250_(1993)_(1-27-2026).csv"

# Output directory
OUTPUT_DIR = "../../data/ground_truth"

# Year to filter EDGAR corpus (based on ground truth file)
YEAR = 1993

## Step 1: Load Ground Truth CSV

In [ ]:
# Load the ground truth file
gt_df = pd.read_csv(GROUND_TRUTH_PATH)
print(f"Loaded ground truth with {len(gt_df)} rows")
print(f"Columns: {list(gt_df.columns)}")
gt_df.head()

## Step 2: Fill NA values with "NULL"

In [ ]:
import numpy as np

gt_df = gt_df.replace(["NULL", ""], np.nan)

print("Replaced 'NULL' and empty strings with NaN")
print(f"NaN counts per column:\n{gt_df.isna().sum()}")
gt_df.head()

## Step 3: Load EDGAR Corpus from HuggingFace and Append Sections

In [ ]:
# Load the EDGAR corpus (streaming for memory efficiency)
print(f"Loading EDGAR corpus from HuggingFace...")
edgar_dataset = load_dataset(
    "c3po-ai/edgar-corpus",
    "default",
    split="train",
    streaming=True,
    revision="refs/convert/parquet",
)
print("Dataset loaded in streaming mode")

# Define section columns to extract
section_cols = [
    "section_1", "section_1A", "section_1B", "section_2", "section_3",
    "section_4", "section_5", "section_6", "section_7", "section_7A",
    "section_8", "section_9", "section_9A", "section_9B", "section_10",
    "section_11", "section_12", "section_13", "section_14", "section_15"
]

# Create a set of filenames we need from ground truth
needed_filenames = set(gt_df['filename'].tolist())
print(f"Looking for {len(needed_filenames)} specific files from ground truth...")

# Only extract documents that match our ground truth filenames
edgar_data = []
found_count = 0
for doc in edgar_dataset:
    # Only process if this filename is in our ground truth
    if doc.get("filename") in needed_filenames:
        row = {"filename": doc["filename"]}
        for col in section_cols:
            row[col] = doc.get(col, "")
        edgar_data.append(row)
        found_count += 1
        
        # Stop once we've found all needed files
        if found_count >= len(needed_filenames):
            print(f"Found all {found_count} required files, stopping early.")
            break
        
        # Progress update every 50 files
        if found_count % 50 == 0:
            print(f"  Found {found_count}/{len(needed_filenames)} files...")

edgar_df = pd.DataFrame(edgar_data)
print(f"Extracted {len(edgar_df)} filings matching ground truth")
print(f"Columns: {list(edgar_df.columns)}")
edgar_df.head()

In [ ]:
# Create a mapping from CIK to sections
# The EDGAR corpus uses 'cik' column
section_cols = [col for col in edgar_df.columns if col.startswith('section_')]
print(f"Section columns found: {section_cols}")

# Merge on filename directly - both datasets have this column
gt_df_merged = gt_df.merge(
    edgar_df[['filename'] + section_cols],
    on='filename',
    how='left',
    suffixes=('', '_edgar')
)

print(f"Merged DataFrame shape: {gt_df_merged.shape}")
print(f"New columns: {list(gt_df_merged.columns)}")
print(f"Merge success rate: {(~gt_df_merged[section_cols[0]].isna()).sum()}/{len(gt_df_merged)} rows matched")

## Step 4: Show Bar Graphs of Non-NULL Values per Category

In [ ]:
# Calculate non-NULL counts for each column
def count_non_null(series):
    """Count values that are not 'NULL' or empty."""
    return ((series != "NULL") & (series != "") & (series.notna())).sum()

# Get counts for all columns (excluding filename and section columns for truth values)
truth_cols = [col for col in gt_df_merged.columns if col.endswith('_truth')]
non_null_counts = {col: count_non_null(gt_df_merged[col]) for col in truth_cols}

print("Non-NULL counts per truth column:")
for col, count in non_null_counts.items():
    print(f"  {col}: {count}/{len(gt_df_merged)} ({count/len(gt_df_merged)*100:.1f}%)")

In [ ]:
# Create bar chart for truth columns
fig, ax = plt.subplots(figsize=(12, 6))

columns = list(non_null_counts.keys())
counts = list(non_null_counts.values())

# Create shorter labels by removing '_truth' suffix
labels = [col.replace('_truth', '') for col in columns]

bars = ax.bar(labels, counts, color='steelblue', edgecolor='black')

# Add value labels on top of bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax.annotate(f'{count}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Non-NULL Count', fontsize=12)
ax.set_title('Non-NULL Values per Ground Truth Category', fontsize=14)
ax.set_ylim(0, len(gt_df_merged) * 1.15)  # Add space for labels

# Add horizontal line for total count
ax.axhline(y=len(gt_df_merged), color='red', linestyle='--', alpha=0.7, label=f'Total rows: {len(gt_df_merged)}')
ax.legend()

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Also show section columns non-NULL counts
section_cols_in_merged = [col for col in gt_df_merged.columns if col.startswith('section_')]
section_non_null_counts = {col: count_non_null(gt_df_merged[col]) for col in section_cols_in_merged}

print("\nNon-NULL counts per section column:")
for col, count in section_non_null_counts.items():
    print(f"  {col}: {count}/{len(gt_df_merged)} ({count/len(gt_df_merged)*100:.1f}%)")

In [ ]:
# Create bar chart for section columns
if section_non_null_counts:
    fig, ax = plt.subplots(figsize=(14, 6))

    columns = list(section_non_null_counts.keys())
    counts = list(section_non_null_counts.values())

    bars = ax.bar(columns, counts, color='forestgreen', edgecolor='black')

    # Add value labels on top of bars
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.annotate(f'{count}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

    ax.set_xlabel('Section', fontsize=12)
    ax.set_ylabel('Non-NULL Count', fontsize=12)
    ax.set_title('Non-NULL Values per SEC 10-K Section', fontsize=14)
    ax.set_ylim(0, len(gt_df_merged) * 1.15)

    # Add horizontal line for total count
    ax.axhline(y=len(gt_df_merged), color='red', linestyle='--', alpha=0.7, label=f'Total rows: {len(gt_df_merged)}')
    ax.legend()

    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Step 5: Save Cleaned CSV with Timestamp

In [ ]:
# Generate timestamp for filename
timestamp = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
output_filename = f"cleaned_EDGAR_gt_({timestamp}).csv"
output_path = os.path.join(OUTPUT_DIR, output_filename)

# Save the cleaned DataFrame
gt_df_merged.to_csv(output_path, index=False)
print(f"Saved cleaned ground truth to: {output_path}")
print(f"Total rows: {len(gt_df_merged)}")
print(f"Total columns: {len(gt_df_merged.columns)}")

In [ ]:
# Summary
print("\n" + "="*50)
print("CLEANUP SUMMARY")
print("="*50)
print(f"Input file: {GROUND_TRUTH_PATH}")
print(f"Output file: {output_path}")
print(f"Total rows: {len(gt_df_merged)}")
print(f"Original columns: {len(gt_df.columns)}")
print(f"Final columns: {len(gt_df_merged.columns)}")
print(f"Section columns added: {len(section_cols_in_merged)}")
print("="*50)